In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
data=pd.read_csv("../input/samplcsv/sample-2.csv")

In [3]:
data

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-007676813_00577,Complete,https://transcription.si.edu/transcribe/32203/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32203/image/NMAAHC-007676813_00577.jpg,1,"8 E encl\r\n\r\nI Anne H. Elliott, widow do so...",/32203/transcriptions/NMAAHC-007676813_00577.txt,\N,\N,32203,Oaths/Pardons,NaN,NaN
1,NMAAHC-007676818_00567,Complete,https://transcription.si.edu/transcribe/32796/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32796/image/NMAAHC-007676818_00567.jpg,1,I E C Whaley solemnly swear that I have carefu...,/32796/transcriptions/NMAAHC-007676818_00567.txt,\N,\N,32796,Oaths/Pardons,NaN,NaN
2,NMAAHC-007676815_00488-000001,Complete,https://transcription.si.edu/transcribe/32218/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32218/image/NMAAHC-007676815_00488-000001.jpg,1,43M\r\nI Mrs E Montgomery solemnly Swear that ...,/32218/transcriptions/NMAAHC-007676815_00488-0...,\N,\N,32218,Oaths/Pardons,NaN,NaN
3,NMAAHC-007677400_00375,Complete,https://transcription.si.edu/transcribe/26149/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/26149/image/NMAAHC-007677400_00375.jpg,1,"No. 2324\r\nI do solemnly swear or affirm, in ...",/26149/transcriptions/NMAAHC-007677400_00375.txt,\N,\N,26149,Oaths/Pardons,NaN,NaN
4,NMAAHC-007677378_00480,Complete,https://transcription.si.edu/transcribe/27345/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/27345/image/NMAAHC-007677378_00480.jpg,1,August 24. 1865\r\nAPPLICATION AND CERTIFICATE...,/27345/transcriptions/NMAAHC-007677378_00480.txt,\N,\N,27345,Oaths/Pardons,NaN,NaN
5,NMAAHC-007676819_00216,Complete,https://transcription.si.edu/transcribe/33018/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/33018/image/NMAAHC-007676819_00216.jpg,1,I Wm Whaley solemnly swear that I have careful...,/33018/transcriptions/NMAAHC-007676819_00216.txt,\N,\N,33018,Oaths/Pardons,NaN,NaN
6,NMAAHC-007676816_00124,Complete,https://transcription.si.edu/transcribe/32224/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32224/image/NMAAHC-007676816_00124.jpg,1,I J F Steinmeyer solemnly swear that I have ca...,/32224/transcriptions/NMAAHC-007676816_00124.txt,\N,\N,32224,Oaths/Pardons,NaN,NaN
7,NMAAHC-007676819_00592,Complete,https://transcription.si.edu/transcribe/33019/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/33019/image/NMAAHC-007676819_00592.jpg,1,I Joseph Edings solemnly swear that I have car...,/33019/transcriptions/NMAAHC-007676819_00592.txt,\N,\N,33019,Oaths/Pardons,NaN,NaN
8,NMAAHC-007677378_00303,Complete,https://transcription.si.edu/transcribe/27345/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/27345/image/NMAAHC-007677378_00303.jpg,1,Vicksburg Miss. \r\nJuly 22 1867 \r\n\r\nThoma...,/27345/transcriptions/NMAAHC-007677378_00303.txt,\N,\N,27345,Oaths/Pardons,NaN,NaN
9,NMAAHC-007676816_00259,Complete,https://transcription.si.edu/transcribe/32224/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/32224/image/NMAAHC-007676816_00259.jpg,1,"455\r\n(RETAINED COPY.)\r\nI, Jacob Schroder. ...",/32224/transcriptions/NMAAHC-007676816_00259.txt,\N,\N,32224,Oaths/Pardons,NaN,NaN


In [4]:
data["transcription_text"][2]

'43M\r\nI Mrs E Montgomery solemnly Swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, \r\nSgd in Montgomery\r\n\r\nSworn to and subscribed before me at Charleston SC this 31 day of Oct 1865.\r\nLevi Stuber\r\nMajor 47th Pa. Vet. Vols. and Provost Marshal.\r\n\r\n[[left margin]] Duplicate [[/left margin]]\r\n\r\nNo.1984\r\nUnited States of America.\r\nI Mrs E. Montgomery of the [[strikethrough]] County [[/strikethrough]]mnly Swear in the presence of Almighty God, that I will henceforth faithfully support and defend the constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves

In [5]:
# text[2]

In [6]:
data["transcription_text"][3]

'No. 2324\r\nI do solemnly swear or affirm, in the presence of Almighty God, that I will henceforth faithfully defend the Constitution of the United States, and the Union of States thereunder, and that I will in like manner abide by and faithfully support all Laws and Proclamations which have been made during the existing rebellion, with reference to the emancipation of slaves.\r\nSO HELP ME GOD,\r\nE.B. Towne\r\n\r\n[[stamp]] INTER.REVENUE 5 5 FIVE IMAGE CENTS 5 5 INLDEXCHANGE [[stamp]]\r\n\r\nSworn to and Subscribed before me, this 31st day of August A.D. 1865.\r\nHu Kennedy\r\nMAYOR\r\nOf the City of New Orleans.'

In [7]:
# text[3]

In [8]:
data["transcription_text"][4]

'August 24. 1865\r\nAPPLICATION AND CERTIFICATE OF\r\nW.A. Fleitas\r\nAmnesty Oath Administered\r\nAt Pass Christian Mississippi\r\nBy J.B. Hiserodt, \r\n66" U.S. Cold. Infty.'

In [9]:
data["transcription_text"][5]

'I Wm Whaley solemnly swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, except\r\n\r\nWilliam Whaley\r\nSworn to and subscribed before me at Charleston SC this 3 day of August 1865.\r\n\r\nLevi Stuber\r\nMay 4 Pa Vols\r\nProv. Marshal \r\nCity of Charleston\r\n\r\n[[left margin]]\r\nDuplicate\r\n[[/left margin]]\r\n\r\nNo. 38 \r\nUnited States of America. \r\n\r\n1 Wm Whaley of the County of Charleston State of So Carolina do solemnly swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the State thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation

In [10]:
# text[5]

In [11]:
data["transcription_text"][6]

'I J F Steinmeyer solemnly swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, \r\nJ.F. Steinmeyer\r\nSworn to and subscribed before me at Charleston Sc this 16th day of Sept 1865.\r\nLevi Stuber\r\nMaj 47 & Pro Marshal\r\n\r\n[[left margin]]Duplicate[[/left margin]]\r\nNo. 1474\r\nUnited States of America.\r\nI J F Steinmeyer of the [[crossed out]] County [[/crossed out]]o solemnly swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of Slaves-So HELP ME GOD.\r\nJ.F. Steinmeyer\r\nSworn

In [12]:
data["transcription_text"][7]

'I Joseph Edings solemnly swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, \r\nJoseph Edings\r\n\r\nSworn and subscribed before me at Orangeburg SC this 16 day of October 1865.\r\nFrank Geise\r\n1st Lt. 54th NYVV\r\nAsst Prov Marshal\r\n\r\nNo.\r\nUnited States of America.\r\nI Joseph Edings of [[crossed-out]] the County of [[/crossed-out]] Orangeburg Dist. State of South Carolina do solemnly swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the Emancipation of slaves-SO HELP ME GOD.\r\nJoseph Edin

In [13]:
data["transcription_text"][8]

'Vicksburg Miss. \r\nJuly 22 1867 \r\n\r\nThomas, Andrew \r\n\r\nOath of Loyalty \r\n\r\nRecd Bureau R.F. & AL. Miss\r\nJuly 22 1867'

In [14]:
data["transcription_text"][9]

'455\r\n(RETAINED COPY.)\r\nI, Jacob Schroder. do solemnly swear, that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON President of the United States of America, on May 29th, 1865, and that I am not expected from the benefits of that Proclamation by any one of the fourteen exceptions therein made, except \r\nJacob Schroder\r\nSworn to and subscribed before me, at Walhalla this 6" day of Nov" 1865.\r\nR A Bray\r\n Capt & Ass\'t. Provost Marshal.\r\n\r\nNo.2\r\nUNITED STATES OF AMERICA.\r\nI, Jacob Schroder of Walhalla State of South Carolina, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all Laws and Proclamations which have been made during the existing Rebellion with reference to the Emancipation of Slaves---"SO HELP ME GOD."\r\nJacob Schroder\r\n\r\nSworn to and su

In [15]:
data["transcription_text"][10]

'[[image - eagle]]\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics [[blank]]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly adm

In [16]:
data["transcription_text"][11]

'OATH.\r\nI, P. P. Bergevin, \r\ndo solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought, not accepted, nor attempted to exercise the functions of any office whatever, under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power, or constitution within the United States, hostile or inimical thereto.  And I do further swear, or affirm, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign and domestic; that I will bear true faith and allegiance to the same; that I take this obligation freely, without any mental reservation or purpose of evasion; and that I will well and faithfully d

In [17]:
data["transcription_text"][12]

'Frank Conner '

In [18]:
data["transcription_text"][13]

'I Caroline Belitzer solemnly Swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, \r\ncaroline Belitzer\r\n\r\nSworn to and subscribed before me at Charleston S.C. this 7th day of September 186 [strikethrough]]5[[/strikethrough]] 6.\r\nF. K. Field\r\nCapt 128th USCT \r\nProv. Mar. \r\n\r\n[[left margin]] Copy F. K. Field\r\nCapt 128" USCT Pro. Mar. [[/left margin]]\r\n\r\nNO. 117\r\nUnited States of America.\r\n\r\nI Caroline Belitzer of the [[strikethrough]] County [[/strikethrough]] dist. of Charleston State of So Carolina do solemnly Swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws 

In [19]:
txte= re.sub("([\(\[]).*?([\)\]])","",data["transcription_text"][10])
txte

']\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics ]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to J.W. Baxter

In [20]:
txte= re.sub("([\(\[]).*?([\)\]])","",data["transcription_text"][10])
txte

']\r\nAMNESTY OATH.\r\n\r\nMississippi City Miss Sept 29 1865.\r\n\r\nI, J.W. Baxter, do solemnly swear, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. SO HELP ME GOD.\r\nJ.W. Baxter\r\n\r\nSworn to and subscribed at Mississippi City\r\nthis 29th day of September 1865, before me\r\nJ. H. Mathews Capt 66th\r\nU.S Col Infty & Provost Marshal.  Sub port Miss\r\n\r\nDESCRIPTION:\r\nAge 24 years\r\nHeight 5ft 8 inches\r\nEyes Blue\r\nColor of Hair light\r\nCharacteristics ]\r\n\r\nHEADQUARTERS, Mississippi City Miss\r\nSept 29th 1865.\r\n\r\nI hereby certify, that the Amnesty Oath prescribed by the Proclamation of the President of the United States, was duly administered to J.W. Baxter

# Preprocessing

In [21]:
text=data["transcription_text"]

In [22]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [23]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [25]:
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:01<00:00, 11.74it/s]


In [26]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 2318.26it/s]


In [27]:
text[1]

'I E C Whaley solemnly swear I carefully read Amnesty Proclamation issued Andrew Johnson President of United State of America May 29th 1865 I excepted benefit of Proclamation one of fourteen exception therein made except E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben Maj 47th PM United States of America E C Whaley of city county Charleston Sate of South Ca solemnly swear presence of Almighty God I henceforth faithfully support defend Constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference Emancipation of SlavesSO HELP ME GOD E C Whaley Sworn subscribed Charleston SC 28 day of Sept 1865 Levi Steuben May 47 PM The fair complexion light hair gray eye 5 foot 10 inch high 39 year profession planter Copy Levi Steuben Maj 47th Pro Marshal City of Charleston'

In [28]:
data["transcription_text"][2]

'43M I Mrs E Montgomery solemnly Swear I carefully read Amnesty Proclamation issued ANDREW JOHNSON President of United States of America May 29th 1865 I excepted benefit of Proclamation one of fourteen exception therein made Sgd Montgomery Sworn subscribed Charleston SC 31 day of Oct 1865 Levi Stuber Major 47th Pa Vet Vols Provost Marshal Duplicate No1984 United States of America I Mrs E Montgomery of County mnly Swear presence of Almighty God I henceforth faithfully support defend constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference Emancipation of Slaves So HELP ME GOD Sgd Montgomery Sworn subscribed Charleston SC 31 day of Oct 1865 Levi Stuber Major 47th Pa Vet Vols Provost Marshal The Fair complexion Dark hair dark eye 5 foot 4 inch high 61 year profession Widow'

In [29]:
text[2]

'43M I Mrs E Montgomery solemnly Swear I carefully read Amnesty Proclamation issued ANDREW JOHNSON President of United States of America May 29th 1865 I excepted benefit of Proclamation one of fourteen exception therein made Sgd Montgomery Sworn subscribed Charleston SC 31 day of Oct 1865 Levi Stuber Major 47th Pa Vet Vols Provost Marshal Duplicate No1984 United States of America I Mrs E Montgomery of County mnly Swear presence of Almighty God I henceforth faithfully support defend constitution of United States Union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference Emancipation of Slaves So HELP ME GOD Sgd Montgomery Sworn subscribed Charleston SC 31 day of Oct 1865 Levi Stuber Major 47th Pa Vet Vols Provost Marshal The Fair complexion Dark hair dark eye 5 foot 4 inch high 61 year profession Widow'

In [30]:
text[3]

'No 2324 I solemnly swear affirm presence of Almighty God I henceforth faithfully defend Constitution of United States Union of States thereunder I like manner abide faithfully support Laws Proclamations made existing rebellion reference emancipation of slave SO HELP ME GOD EB Towne INTERREVENUE 5 5 FIVE IMAGE CENTS 5 5 INLDEXCHANGE Sworn Subscribed 31st day of August AD 1865 Hu Kennedy MAYOR Of City of New Orleans'

In [31]:
text[4]

'August 24 1865 APPLICATION AND CERTIFICATE OF WA Fleitas Amnesty Oath Administered At Pass Christian Mississippi By JB Hiserodt 66 US Cold Infty'

In [32]:
text[10]

'AMNESTY OATH Mississippi City Miss Sept 29 1865 I JW Baxter solemnly swear presence of Almighty God I henceforth faithfully support protect defend Constitution of United States union of States thereunder I like manner abide faithfully support law proclamation made existing rebellion reference emancipation of slave SO HELP ME GOD JW Baxter Sworn subscribed Mississippi City 29th day of September 1865 J H Mathews Capt 66th US Col Infty Provost Marshal Sub port Miss DESCRIPTION Age 24 year Height 5ft 8 inch Eyes Blue Color of Hair light Characteristics HEADQUARTERS Mississippi City Miss Sept 29th 1865 I hereby certify Amnesty Oath prescribed Proclamation of President of United States duly administered JW Baxter of Harrison Co order 29th day of Sept 29 1865 Frank E Miller Maj 66th USCI Commanding'

# spacy

In [33]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-

In [34]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [35]:
L1=[]
def NER (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L1.append([(word.text, word.start_char, word.end_char, word.label_,spacy.explain(word.label_)) for word in nlp_text.ents])
        
NER(text)
l0=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
for i in range(len(L1)):
    for j in range(len(L1[i])):
        l0.append(L1[i][j][0])
        l2.append(L1[i][j][1])
        l3.append(L1[i][j][2])
        l4.append(L1[i][j][3])
        l5.append(L1[i][j][4])

df1=pd.DataFrame({"entities":l0,"start_char":l2,"end_char":l3 ,"label": l4, "explain": l5, "frequency":""})
df1.head()

,entities,start_char,end_char,label,explain,frequency
0,Anne H Elliott,11,25,PERSON,"People, including fictional",
1,Amnesty,54,61,ORG,"Companies, agencies, institutions, etc.",
2,Andrew Johnson,82,96,PERSON,"People, including fictional",
3,United States of America,110,134,GPE,"Countries, cities, states",
4,May 29th 1865,135,148,DATE,Absolute or relative dates or periods,


In [36]:
Person=[]
ORG=[]
DATE=[]
GPE=[]

for i in range(df1.shape[0]):
    if df1.label.loc[i]=="PERSON":
        Person.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="ORG" :
        ORG.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="DATE" :
        DATE.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="GPE" :
        GPE.append(df1.entities.loc[i])
GPE=np.array(GPE)
ORG=np.array(ORG)
Person=np.array(Person)
DATE=np.array(DATE)

In [37]:
DATE

array(['May 29th 1865', '27th day', '1865', '27th day', '1865', '63 year',
       'May 29th 1865', '28 day', '28 day', 'May 47 PM', '39 year',
       'May 29th 1865', '31 day', '1865', 'No1984', '31 day', '1865',
       '31st day', 'August 24 1865', 'May 29th 1865',
       '3 day of August 1865', 'May 4', '3 day', '1865', 'May 29th 1865',
       '16th day', '1474', '16th day', 'May 29th 1865',
       '16 day of October 1865', '16 day of October 1865', 'July 22',
       'July 22', '1867', 'May 29th 1865', '6 day', '6 day',
       '29th day of September 1865', 'Age 24 year', 'Sept 29th 1865',
       '29th day', 'May 29th 1865', '7th day of September',
       '7th day of September', '20th day of August 1865',
       '20th day of August 1865', 'August 29th 1865',
       '22nd day of August 1865', 'Miss 31st day', '1865 1865', '31 day',
       '1865'], dtype='<U26')

In [38]:
len(Person)

41

In [39]:
Person

array(['Anne H Elliott', 'Andrew Johnson', 'Hazel', 'Whaley',
       'Andrew Johnson', 'Copy Levi Steuben', 'Hu Kennedy',
       'Amnesty Oath Administered At', 'Hiserodt', 'William Whaley Sworn',
       'William Whaley Sworn', 'Brown', 'Charleston Sc',
       'JF Steinmeyer Sworn', 'Charleston Sc', 'Gry', 'Joseph Edings',
       'Thomas Andrew Oath of', 'Jacob Schroder Sworn', 'Walhalla',
       'Walhalla', 'Bray Lieut', 'Laborer', 'Characteristics',
       'Amnesty Oath', 'Woodville Wilkinson', 'Frank Conner',
       'Triplicate AMNESTY', 'Hiram Smith', 'Hiram Smith',
       'Pass Christian', 'Captn', 'Amnesty Oath', 'Hiram Smith',
       'Captn Commanding', 'Robert A',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Gibbs Henderson', 'City Miss', 'Amnesty Oath', 'Frank E Miller'],
      dtype='<U58')

# Hugging face  : Flair

**ner-english** 

In [40]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.00 MiB/s, done.


In [41]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 07:50:44,850 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-08 07:51:03,847 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [42]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-08 07:51:21,262 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-08 07:51:23,898 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [43]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 07:52:35,594 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-08 07:52:50,806 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [44]:
from flair.data import Sentence
from flair.models import SequenceTagger
T2=[]
D2={}
for i in range(20):
        sentencess =Sentence(text[i])
        tagger.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T2.append(entity)
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                         D2[entity.text]= entity.get_label("ner").value

In [45]:
T3=[]
D3={}

for i in range(20):
        sentencess =Sentence(text[i])
        tagger1.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T3.append(entity)
                     if (entity.get_label("ner").score > 0.90)  :
                         D3[entity.text]= entity.get_label("ner").value

In [46]:
T4=[]
D4={}

for i in range(20):
        sentencess =Sentence(text[i])
        tagger2.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T4.append(entity)
                     if (entity.get_label("ner").score > 0.90)  :
                         D4[entity.text]= entity.get_label("ner").value

In [47]:
# lst=[]
# dic={}

# for i in range(20):
#         sentencess =Sentence(text[i])
#         tagger2.predict(sentencess)
#         for entity in sentencess.get_spans('ner'):
#                      T4.append(entity)
#                      if (entity.get_label("ner").score > 0.90)  :
#                          D4[entity.text]= entity.get_label("ner").value

In [48]:
len(D2)

104

In [49]:
len(D3)

22

In [50]:
len(D4)

108

In [51]:
ent=[]
label=[]
for i in D2 :
    ent.append(i)
    label.append(D2[i])

In [52]:
ent1=[]
label1=[]
for i in D3 :
    ent1.append(i)
    label1.append(D3[i])

In [53]:
ent2=[]
label2=[]
for i in D4 :
    ent2.append(i)
    label2.append(D4[i])

In [54]:
final=pd.DataFrame({"entities":ent, "label":label})

In [55]:
final1=pd.DataFrame({"entities":ent1, "label":label1})

In [56]:
final2=pd.DataFrame({"entities":ent2, "label":label2})

In [57]:
P=[]
OR=[]
GP=[]
Date=[]
LOC=[]
for i in range(len(D2)):
    if final.label.loc[i]=="PERSON":
        P.append(final.entities.loc[i])
    elif final.label.loc[i]=="ORG":
        OR.append(final.entities.loc[i])
    elif final.label.loc[i]=="GPE":
        GP.append(final.entities.loc[i])
    elif final.label.loc[i]=="DATE":
        Date.append(final.entities.loc[i])
    elif final.label.loc[i]=="LOC":
        LOC.append(final.entities.loc[i])
        

OR=np.array(OR)
P=np.array(P)
LOC=np.array(LOC)
Date=np.array(Date)
GP=np.array(GP)



In [58]:
Date

array(['May 29th 1865', '27th day of Novb 1865', '27th day of Novbr 1865',
       '63 year', '28 day of Sept 1865', 'Maj 47th PM', 'May 47 PM',
       '39 year', 'Maj 47th', '31 day of Oct 1865', '61 year',
       '31st day of August AD 1865', 'August 24 1865',
       '3 day of August 1865', 'May 4', '3 day of Augt 1865', 'Maj 47',
       '48 year', '16th day of Sept 1865', '58 year',
       '16 day of October 1865', 'twenty eight year', 'July 22',
       '6 day of Nov 1865', '58 year of age', 'Sept 29 1865',
       '29th day of September 1865', '24 year', 'Sept 29th 1865',
       '29th day of Sept 29 1865', '3d day of February 1868',
       '7th day of September 186', 'Aug 20 1865',
       '20th day of August 1865', 'Thirty nine', 'Aug 20th 1865',
       'August 29th 1865', '22nd day of August 1865', '18th Sept 1865',
       'Aug 31 1865', '31st day of Aug 1865', '26 Years',
       '31 day of Aug 1865'], dtype='<U26')

In [59]:
P1=[]
OR1=[]
LOC1=[]
for i in range(len(D3)):
    if final1.label.loc[i]=="PER":
        P1.append(final1.entities.loc[i])
    elif (final1.label.loc[i]=="ORG") | (final1.label.loc[i]=="MISC"):
        OR1.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="LOC" :
        LOC1.append(final1.entities.loc[i])
        

OR1=np.array(OR1)
P1=np.array(P1)
LOC1=np.array(LOC1)


In [60]:
P2=[]
OR2=[]
LOC2=[]
for i in range(len(D4)):
    if final2.label.loc[i]=="PER":
        P2.append(final2.entities.loc[i])
    elif (final2.label.loc[i]=="ORG") | (final2.label.loc[i]=="MISC"):
        OR2.append(final2.entities.loc[i])
    elif final2.label.loc[i]=="LOC" :
        LOC2.append(final2.entities.loc[i])
        

OR2=np.array(OR2)
P2=np.array(P2)
LOC2=np.array(LOC2)


In [61]:
len(Person)

41

In [62]:
len(P)

40

In [63]:
len(P1)

15

In [64]:
len(P2)

45

In [65]:
np.unique(Person)

array(['Amnesty Oath', 'Amnesty Oath Administered At', 'Andrew Johnson',
       'Anne H Elliott', 'Bray Lieut', 'Brown', 'Captn',
       'Captn Commanding', 'Characteristics', 'Charleston Sc',
       'City Miss', 'Copy Levi Steuben', 'Frank Conner', 'Frank E Miller',
       'Gibbs Henderson', 'Gry', 'Hazel', 'Hiram Smith', 'Hiserodt',
       'Hu Kennedy', 'JF Steinmeyer Sworn', 'Jacob Schroder Sworn',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Joseph Edings', 'Laborer', 'Pass Christian', 'Robert A',
       'Thomas Andrew Oath of', 'Triplicate AMNESTY', 'Walhalla',
       'Whaley', 'William Whaley Sworn', 'Woodville Wilkinson'],
      dtype='<U58')

In [66]:
np.unique(P)

array(['ANDREW JOHNSON', 'Aiken Burnwell', 'Andrew Johnson',
       'Anne H Elliott', 'Bergevin', 'Caroline Belitzer', 'City', 'E',
       'E C Whaley', 'Frank Conner', 'Frank E Miller', 'Frank Geise',
       'Geo D Carpenter', 'Hiram Smith', 'Holwell', 'Hu Kennedy',
       'I E C Whaley', 'I J F Steinmeyer', 'J E Thuring', 'J H Mathews',
       'JB Hiserodt', 'JF Steinmeyer', 'JH Mathews', 'JW Baxter',
       'Jacob Schroder', 'Jacob Schroder Sworn', 'James D Kendall',
       'James E Thuring', 'Joseph Edings', 'Levi Steuben', 'Levi Stuber',
       'Levi Stuber Major', 'P H Joor', 'Robert A Wilson',
       'S Wilde Durson', 'Vicksburg', 'Whaley', 'William Whaley Sworn',
       'caroline Belitzer', 'mark Sworn'], dtype='<U20')

In [67]:
np.unique(P1)

array(['Andrew Johnson', 'Anne H', 'Anne H Elliott', 'Caroline Belitzer',
       'E Montgomery', 'Frank Conner', 'Hiram Smith', 'J H Mathews',
       'Jacob Schroder', 'James E Thuring', 'Joseph Edings', 'Marshal',
       'Robert A Wilson', 'Sworn', 'Wm Whaley'], dtype='<U17')

In [68]:
np.unique(P2)

array(['ANDREW JOHNSON', 'Almighty', 'Almighty God', 'Andrew Johnson',
       'Anne H Elliott', 'Caroline Belitzer', 'E C Whaley', 'EB Towne',
       'Frank Conner', 'Frank E Miller', 'Frank Geise', 'GOD',
       'Geo D Carpenter', 'God', 'H Elliott', 'H F Holwell',
       'Hiram Smith', 'Hu Kennedy', 'I E C Whaley', 'I J F Steinmeyer',
       'J E Thuring', 'J H Mathews', 'JB Hiserodt', 'JF Steinmeyer',
       'JH Mathews', 'JW Baxter', 'Jacob Schroder', 'James D Kendall',
       'James E Thuring', 'Joseph Edings', 'Joseph Edings Sworn',
       'Levi Steuben', 'Levi Stuber', 'Mrs E Montgomery', 'P H Joor',
       'P P Bergevin', 'R A Bray', 'Robert A Wilson', 'S Wilde Durson',
       'S Wilde Durson Magt', 'S Wilde Durson Magtr', 'Sgd Montgomery',
       'William Whaley', 'Wm Whaley', 'caroline Belitzer'], dtype='<U20')

****

In [69]:
np.unique(ORG)

array(['AMNESTY', 'ANDREW', 'Amnesty', 'Amnesty Proclamation',
       'Belitzer Sworn', 'Charleston State of So Carolina',
       'Color of Eyes Blue Color of Hair Dark Characteristics',
       'Emancipation of Slaves', 'Eyes Blue Color of Hair',
       'Eyes Blue Color of Hair Light Characteristics', 'H F Holwell of',
       'H Mathews Capt', 'JF Steinmeyer Sworn', 'Joseph Edings Sworn',
       'Lawyer', 'Levi Stuber',
       'Magistrate S Wilde Durson AHE AHE AHE H Elliott',
       'OATH I P P Bergevin',
       'P H Joor Justice of Peace Wilkinson County State of',
       'Rebellion reference Emancipation of Slaves',
       'Regt U S C Infy Provost Marshal', 'Sgd Montgomery Sworn', 'Sworn',
       'UNITED STATES FORCES No 1282 PROVOST MARSHALS OFFICE',
       'US Col Infty Provost Marshal', 'USCI Commanding',
       'United State of America', 'United States Union',
       'United States Union of State', 'Whaley Sworn', 'Wilson'],
      dtype='<U53')

In [70]:
np.unique(OR1)

array(['James D Kendall Amnesty Oath Administered At Oacen Springs Miss 18th',
       'WA Fleitas Amnesty Oath Administered At Pass Christian Mississippi By JB Hiserodt'],
      dtype='<U81')

In [71]:
np.unique(OR2)

array(['128th USCT', '47th Pa', 'AMNESTY OATH', 'Amnesty', 'Amnesty Oath',
       'Amnesty Proclamation', 'Constitution', 'Emancipation of Slaves',
       'Emancipation of SlavesSO', 'Emancipation of SlavesSo',
       'MARSHALS OFFICE', 'NYVV', 'Oath',
       'Thomas Andrew Oath of Loyalty Recd Bureau',
       'Thos Gibbs Henderson Co', 'U S C', 'UNITED STATES FORCES', 'US',
       'US Cold Inf', 'US Cold Infty', 'USCI', 'USCT', 'Union', 'Vols',
       'WA Fleitas Amnesty Oath', 'Walhalla', 'Woodville Wilkinson Co'],
      dtype='<U41')

****

In [72]:
np.unique(LOC)

array(['South Ca'], dtype='<U8')

In [73]:
np.unique(LOC1)

array(['AMERICA', 'So Carolina', 'South Carolina', 'US', 'United States'],
      dtype='<U14')

In [74]:
np.unique(LOC2)

array(['Aiken', 'Aiken Burnwell', 'Charleston', 'Charleston Sate',
       'Charleston Sc', 'City', 'City of Charleston',
       'City of New Orleans', 'County', 'County of Charleston',
       'County of Orangeburg', 'District of Beaufort', 'Gainsville',
       'Harrison Co', 'Marshal City of Charleston', 'Miss', 'Miss City',
       'Mississippi City', 'NC', 'Oacen Springs', 'Orangeburg SC',
       'Pass Christian Mississippi', 'Port of Miss', 'SC', 'South Ca',
       'State of So Carolina', 'State of South Carolina',
       'UNITED STATES OF AMERICA', 'United State of America',
       'United States', 'United States Union of State',
       'United States Union of States', 'United States of America',
       'Walhalla State of South Carolina', 'Warren Co',
       'Wilkinson County State of Mississippi'], dtype='<U37')

****

In [75]:
np.unique(Date)

array(['16 day of October 1865', '16th day of Sept 1865',
       '18th Sept 1865', '20th day of August 1865',
       '22nd day of August 1865', '24 year', '26 Years',
       '27th day of Novb 1865', '27th day of Novbr 1865',
       '28 day of Sept 1865', '29th day of Sept 29 1865',
       '29th day of September 1865', '3 day of Augt 1865',
       '3 day of August 1865', '31 day of Aug 1865', '31 day of Oct 1865',
       '31st day of Aug 1865', '31st day of August AD 1865', '39 year',
       '3d day of February 1868', '48 year', '58 year', '58 year of age',
       '6 day of Nov 1865', '61 year', '63 year',
       '7th day of September 186', 'Aug 20 1865', 'Aug 20th 1865',
       'Aug 31 1865', 'August 24 1865', 'August 29th 1865', 'July 22',
       'Maj 47', 'Maj 47th', 'Maj 47th PM', 'May 29th 1865', 'May 4',
       'May 47 PM', 'Sept 29 1865', 'Sept 29th 1865', 'Thirty nine',
       'twenty eight year'], dtype='<U26')

In [76]:
np.unique(GP)

array(['Charleston', 'Charleston S C', 'Charleston SC',
       'Mississippi City', 'Orangeburg', 'So Carolina', 'South Carolina',
       'UNITED STATES OF', 'US', 'United State of America',
       'United States', 'United States of America'], dtype='<U24')

# Post processing

**For dates**

In [77]:
import regex as re
Pattern1=re.compile(r'^\d\.*.*\d\d$')
Pattern2=re.compile(r'^[A-Z]\.*.*\d \.*.*\d\d$')

In [78]:
M=[]
for i in Date:
        date=re.search(r'^\d\.*.*\d\d$' ,i)
        date2=re.search(r'^[A-Z]\.*.*\d \.*.*\d\d$' ,i)
        M.append(date)
        M.append(date2)
       

In [79]:
Final_date=[]
for i in M:
    if i!=None:
        Final_date.append(i.group())

In [80]:
N=[]
for i in Final_date:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z]\.*.*)\s([0-9]{3,4})',r'\1-\2-\3' ,i)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{3,4})',r'\2-\1-\3' ,i)
        if Final_date1 != i:   
            N.append(Final_date1)
        if Final_date2 != i :
            N.append(Final_date2)

In [81]:
N

['27-Novb-1865',
 '27-Novbr-1865',
 '28-Sept-1865',
 '31-Oct-1865',
 '31-AD-1865',
 '24-August-1865',
 '3-August-1865',
 '3-Augt-1865',
 '16-Sept-1865',
 '16-October-1865',
 '6-Nov-1865',
 '29-Sept-1865',
 '29-September-1865',
 '29-Sept 29-1865',
 '29th day of 29-Sept-1865',
 '3-February-1868',
 '7-September-186',
 '20-Aug-1865',
 '20-August-1865',
 '22-August-1865',
 '18-Sept-1865',
 '31-Aug-1865',
 '31-Aug-1865',
 '31-Aug-1865']

**great !**

# best result obtained 

***Person***

In [82]:
np.unique(P2)

array(['ANDREW JOHNSON', 'Almighty', 'Almighty God', 'Andrew Johnson',
       'Anne H Elliott', 'Caroline Belitzer', 'E C Whaley', 'EB Towne',
       'Frank Conner', 'Frank E Miller', 'Frank Geise', 'GOD',
       'Geo D Carpenter', 'God', 'H Elliott', 'H F Holwell',
       'Hiram Smith', 'Hu Kennedy', 'I E C Whaley', 'I J F Steinmeyer',
       'J E Thuring', 'J H Mathews', 'JB Hiserodt', 'JF Steinmeyer',
       'JH Mathews', 'JW Baxter', 'Jacob Schroder', 'James D Kendall',
       'James E Thuring', 'Joseph Edings', 'Joseph Edings Sworn',
       'Levi Steuben', 'Levi Stuber', 'Mrs E Montgomery', 'P H Joor',
       'P P Bergevin', 'R A Bray', 'Robert A Wilson', 'S Wilde Durson',
       'S Wilde Durson Magt', 'S Wilde Durson Magtr', 'Sgd Montgomery',
       'William Whaley', 'Wm Whaley', 'caroline Belitzer'], dtype='<U20')

****

***Organization***

In [83]:
np.unique(OR2)


array(['128th USCT', '47th Pa', 'AMNESTY OATH', 'Amnesty', 'Amnesty Oath',
       'Amnesty Proclamation', 'Constitution', 'Emancipation of Slaves',
       'Emancipation of SlavesSO', 'Emancipation of SlavesSo',
       'MARSHALS OFFICE', 'NYVV', 'Oath',
       'Thomas Andrew Oath of Loyalty Recd Bureau',
       'Thos Gibbs Henderson Co', 'U S C', 'UNITED STATES FORCES', 'US',
       'US Cold Inf', 'US Cold Infty', 'USCI', 'USCT', 'Union', 'Vols',
       'WA Fleitas Amnesty Oath', 'Walhalla', 'Woodville Wilkinson Co'],
      dtype='<U41')

****

***GPE and location***

In [84]:
np.unique(GPE)

array(['Charleston', 'Gainsville', 'Mississippi', 'Mississippi City',
       'NC', 'New Orleans', 'South Carolina', 'States', 'US',
       'United States', 'United States of America', 'Vicksburg'],
      dtype='<U24')

In [85]:
np.unique(LOC2)

array(['Aiken', 'Aiken Burnwell', 'Charleston', 'Charleston Sate',
       'Charleston Sc', 'City', 'City of Charleston',
       'City of New Orleans', 'County', 'County of Charleston',
       'County of Orangeburg', 'District of Beaufort', 'Gainsville',
       'Harrison Co', 'Marshal City of Charleston', 'Miss', 'Miss City',
       'Mississippi City', 'NC', 'Oacen Springs', 'Orangeburg SC',
       'Pass Christian Mississippi', 'Port of Miss', 'SC', 'South Ca',
       'State of So Carolina', 'State of South Carolina',
       'UNITED STATES OF AMERICA', 'United State of America',
       'United States', 'United States Union of State',
       'United States Union of States', 'United States of America',
       'Walhalla State of South Carolina', 'Warren Co',
       'Wilkinson County State of Mississippi'], dtype='<U37')

****

**Date**

In [86]:
N

['27-Novb-1865',
 '27-Novbr-1865',
 '28-Sept-1865',
 '31-Oct-1865',
 '31-AD-1865',
 '24-August-1865',
 '3-August-1865',
 '3-Augt-1865',
 '16-Sept-1865',
 '16-October-1865',
 '6-Nov-1865',
 '29-Sept-1865',
 '29-September-1865',
 '29-Sept 29-1865',
 '29th day of 29-Sept-1865',
 '3-February-1868',
 '7-September-186',
 '20-Aug-1865',
 '20-August-1865',
 '22-August-1865',
 '18-Sept-1865',
 '31-Aug-1865',
 '31-Aug-1865',
 '31-Aug-1865']

****